In [1]:
import os

In [2]:
print(os.getcwd())

C:\AI-NLP\learn-NLP-luhuibo\lesson-13-RNN-kaggle


In [3]:
!pip install tensorflow

In [4]:
from urllib.request import urlretrieve
url = 'https://download.wetransfer.com//us2/5bb11a202aa730ace4122fb91c3043e320191005083240/316f30ced5f12241389f70c041cfb90b9cf5bb86/toxic_classification.pickle?cf=y&token=eyJhbGciOiJIUzI1NiJ9.eyJleHAiOjE1NzAyNjcxNjIsInVuaXF1ZSI6IjViYjExYTIwMmFhNzMwYWNlNDEyMmZiOTFjMzA0M2UzMjAxOTEwMDUwODMyNDAiLCJmaWxlbmFtZSI6InRveGljX2NsYXNzaWZpY2F0aW9uLnBpY2tsZSIsImhvdCI6ZmFsc2UsImJ5dGVzX2VzdGltYXRlZCI6MzYxMTk5NzU3LCJ3YXliaWxsX3VybCI6Imh0dHA6Ly9wcm9kdWN0aW9uLmJhY2tlbmQuc2VydmljZS51cy1lYXN0LTEud3Q6OTI5Mi93YXliaWxsL3YxL2EwZGI3YzkwZmFkNzUxNWM3ZDA5NzczMTI4NmQzZTIzYzQxYTZlNTBhZmM3NjI1MjQzYjUzODE5YjFmNzcwNDNhMWI5ODBhYTcwM2RlZTVjZjFkYzNkMjk3ZGE2NWZjYzllNDFiMzI1ZDI5NmU1MWE1YWE1MzBiZDNmMTE0OWU1IiwiY2FsbGJhY2siOiJ7XCJmb3JtZGF0YVwiOntcImFjdGlvblwiOlwiaHR0cDovL3Byb2R1Y3Rpb24uZnJvbnRlbmQuc2VydmljZS5ldS13ZXN0LTEud3Q6MzAwMC9hcGkvYmFja2VuZC90cmFuc2ZlcnMvNWJiMTFhMjAyYWE3MzBhY2U0MTIyZmI5MWMzMDQzZTMyMDE5MTAwNTA4MzI0MC9kb3dubG9hZHMvNzQyNTY2MDE2Ny9jb21wbGV0ZWRcIn0sXCJmb3JtXCI6e1wic3RhdHVzXCI6W1wicGFyYW1cIixcInN0YXR1c1wiXSxcImRvd25sb2FkX2lkXCI6XCI3NDI1NjYwMTY3XCJ9fSJ9.5riIV8H7KzWX88yF0xaG3jNTWwFjwdYjYfyQhM90vH8' 
urlretrieve(url, "/content/sample_data/toxic_classification.pickle")

HTTPError: HTTP Error 410: Gone

In [ ]:
import pickle
import os
data_root = '/content/sample_data'
pickle_file = os.path.join(data_root, 'toxic_classification.pickle')

In [ ]:
import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D,concatenate
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

In [ ]:
with open(pickle_file, 'rb') as f:
 save = pickle.load(f)
 x_train = save['x_train']
 y_train = save['y_train']
 x_test = save['x_test']
 y_test = save['y_test']
 embedding_matrix = save['embedding_matrix']
 del save # hint to help gc free up memory

In [ ]:
class RocAucEvaluation(Callback):
 def __init__(self, validation_data=(), interval=1):
 super(Callback, self).__init__()

 self.interval = interval
 self.X_val, self.y_val = validation_data

 def on_epoch_end(self, epoch, logs={}):
 if epoch % self.interval == 0:
 y_pred = self.model.predict(self.X_val, verbose=0)
 score = roc_auc_score(self.y_val, y_pred)
 print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [ ]:
max_features = 40000
maxlen = 200
embed_size = 300

HIDDNE_SIZE_1 = 256
HIDDNE_SIZE_2 = 128

def get_model():
 ###embedding 和textcnn一样处理
 inp = Input(shape=(maxlen, ))
 x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
 x = SpatialDropout1D(0.2)(x)
 
 #conv_0 = Conv1D(num_filters, kernel_size=kernel_size, strides=1)(x_emb)#这里也可以用conv1D，因为在embed_size
 #等于词向量维度大小，故在列方向相当于没有做卷积操作，使用Conv2D的效果和Conv1D一样
 GRU1 = Bidirectional(GRU(HIDDNE_SIZE_1, return_sequences=True, recurrent_dropout= 0.2,
 input_shape=(maxlen, embed_size)))(x)
 
 GRU2 = Bidirectional(GRU(HIDDNE_SIZE_1, return_sequences=False, recurrent_dropout= 0.2,
 input_shape=(maxlen, HIDDNE_SIZE_1)))(GRU1)
 
 z = Dropout(0.2)(GRU2)
 
 fully1 = Dense(HIDDNE_SIZE_2, activation='relu')(z)
 
 outp = Dense(6, activation='sigmoid')(fully1) 
 
 model = Model(inputs=inp, outputs=outp)
 model.compile(loss='binary_crossentropy',
 optimizer='adam',
 metrics=['accuracy'])
 return model

In [ ]:
model = get_model()
batch_size = 256
epochs = 3

X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.9, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
 callbacks=[RocAuc])